In [ ]:
#!pip install mtcnn
import keras
from keras.preprocessing import image as kimage
from keras.applications import mobilenet_v2
from tqdm import tqdm
keras.__version__

from sklearn.neighbors import KDTree
import joblib

from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
from time import time
import os
import mtcnn
from PIL import Image

### Import data and face detection 

In [ ]:
list_name=os.listdir('thumbnails_features_deduped_publish')
maximages=1

In [ ]:
len(list_name)

In [ ]:
# Data loader
def load_data():
    base_path = 'thumbnails_features_deduped_publish/'
    paths = []
    faces = []
  
    for vip in tqdm(list_name):
        base_path_name = base_path + vip
        results=[]
    
        for fi,f in enumerate(sorted(os.listdir(base_path_name))):
            if f.endswith('.jpg') and fi<maximages and results == []:
            
                cur_path = base_path_name + '/' + f
          
                image = kimage.load_img(cur_path, target_size=(224, 224))
                foto = image.convert('RGB')
                # convert to array
                pixels = np.asarray(foto)
                # create the detector, using default weights
                detector = mtcnn.MTCNN()
                # detect faces in the image
                results = detector.detect_faces(pixels)
                
                if results != [] :
                    # extract the bounding box from the first face
                    x1, y1, width, height = results[0]['box']
                    # bug fix
                    x1, y1 = abs(x1), abs(y1)
                    x2, y2 = x1 + width, y1 + height
                    # extract the face
                    face = pixels[y1:y2, x1:x2]
                    # resize pixels to the model size
                    faccia = Image.fromarray(face)
                    faccia = faccia.resize(size=(224,224))
                    face_array = np.asarray(faccia)
                    faces.append(face_array)

                    paths.append(cur_path)
                elif fi>maximages and results==[] :
                    print ('skipped' + vip)

    return faces, paths 

In [ ]:
facce, paths = load_data()

In [ ]:
load_data(facce,paths)

In [ ]:
# Numero di immagini analizzate
print (len(paths), len(facce))

### Feature extraction with MobileNet pre-trained on imagenet

In [ ]:
net = mobilenet_v2.MobileNetV2(include_top=False, weights='imagenet', pooling='max', input_shape=(224, 224, 3))

In [ ]:
def neural_features(img):
    x = kimage.img_to_array(img)
    x = mobilenet_v2.preprocess_input(x)
    x = np.expand_dims(x, axis=0)
    f = net.predict(x)
    return f.flatten()

In [ ]:
def feature_extractor():
    features=[]
    for face in tqdm(facce):
        cur_features = neural_features(face)
        features.append(cur_features)
  
    features = np.array(features)
    return features

In [ ]:
X_train = feature_extractor()

In [ ]:
# Dimensioni matrice di features estratte
X_train.shape

###  KDTree building and exporting

In [ ]:
tree = KDTree(X_train)

In [ ]:
os.getcwd()

In [ ]:
# Salvataggio albero di ricerca
joblib.dump(tree, 'RetrievalTree.joblib')

In [ ]:
# Salvataggio paths immagini
joblib.dump(paths,'Paths.joblib')